<a href="https://colab.research.google.com/github/OffirRokach/ColabNotebook/blob/main/returning_customers_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Link to chatGPT prompts: https://chat.openai.com/share/7b92932e-bf5f-477f-b861-af08d9e015c3
import sqlite3
import pandas as pd  # Import pandas for handling the data in table format

# Connect to SQLite database (it will be created in the current directory if it does not exist)
conn = sqlite3.connect('example.db')
c = conn.cursor()

# Create the table
c.execute('''
CREATE TABLE IF NOT EXISTS sales (
    customer_num INTEGER,
    date_of_purchase DATE,
    item_purchased TEXT,
    cost REAL,
    item_returned BOOLEAN
)
''')

# Data to insert
data = [
    (1234567, '2023-04-01', 'shirt', 20.00, 0),
    (1234567, '2023-04-02', 'shoes', 50.00, 1),
    (2345678, '2023-04-03', 'coat', 80.00, 0),
    (3456789, '2023-04-04', 'pants', 25.00, 0),
    (4567890, '2023-04-05', 'shirt', 20.00, 1),
    (5678901, '2023-04-06', 'shoes', 50.00, 0),
    (6789012, '2023-04-07', 'coat', 80.00, 1),
    (7890123, '2023-04-08', 'pants', 25.00, 0),
    (8901234, '2023-04-09', 'shirt', 20.00, 0),
    (9012345, '2023-04-10', 'shoes', 50.00, 1),
    (6789012, '2023-04-11', 'coat', 80.00, 0),
    (5678901, '2023-04-12', 'pants', 25.00, 1),
    (1234567, '2023-04-13', 'pants', 40.00, 0)
]

# Insert data into the table
c.executemany('INSERT INTO sales VALUES (?,?,?,?,?)', data)

# Commit the changes
conn.commit()

# Query the table using pandas for nicely formatted output
df = pd.read_sql_query('SELECT * FROM sales', conn)
print(df)

# Close the connection
conn.close()



In [14]:
import sqlite3
import pandas as pd  # Import pandas for handling the data in table format

# Connect to SQLite database
conn = sqlite3.connect('example.db')
c = conn.cursor()

# Drop the existing table if needed, and recreate it
c.execute('DROP TABLE IF EXISTS sales')
c.execute('''
CREATE TABLE sales (
    customer_num INTEGER,
    date_of_purchase DATE,
    item_purchased TEXT,
    cost REAL,
    item_returned BOOLEAN
)
''')

# Data to insert
data = [
    (1234567, '2023-04-01', 'shirt', 20.00, 0),
    (1234567, '2023-04-02', 'shoes', 50.00, 1),
    (2345678, '2023-04-03', 'coat', 80.00, 0),
    (3456789, '2023-04-04', 'pants', 25.00, 0),
    (4567890, '2023-04-05', 'shirt', 20.00, 1),
    (5678901, '2023-04-06', 'shoes', 50.00, 0),
    (6789012, '2023-04-07', 'coat', 80.00, 1),
    (7890123, '2023-04-08', 'pants', 25.00, 0),
    (8901234, '2023-04-09', 'shirt', 20.00, 0),
    (9012345, '2023-04-10', 'shoes', 50.00, 1),
    (6789012, '2023-04-11', 'coat', 80.00, 0),
    (5678901, '2023-04-12', 'pants', 25.00, 1),
    (1234567, '2023-04-13', 'pants', 40.00, 0)
]

# Insert data into the table
c.executemany('INSERT INTO sales VALUES (?,?,?,?,?)', data)

# Commit the changes
conn.commit()

# Query the table to find returning customers first way
df = pd.read_sql_query('SELECT customer_num, COUNT(*) as num_purchases FROM sales GROUP BY customer_num HAVING COUNT(*) > 1', conn)
print(df)

# Close the connection
conn.close()

   customer_num  num_purchases
0       1234567              3
1       5678901              2
2       6789012              2


In [16]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('example.db')

# SQL query to find returning customers second way
query = """
SELECT customer_num,
       date_of_purchase,
       item_purchased,
       cost,
       item_returned,
       CASE
           WHEN ROW_NUMBER() OVER (PARTITION BY customer_num ORDER BY date_of_purchase) > 1 THEN 0
           ELSE 1
       END AS first_purchase_indicator
FROM sales
"""

# Execute the query and use pandas to display the results in a table format
df = pd.read_sql_query(query, conn)
print(df)

# Close the connection
conn.close()


    customer_num date_of_purchase item_purchased  cost  item_returned  \
0        1234567       2023-04-01          shirt  20.0              0   
1        1234567       2023-04-02          shoes  50.0              1   
2        1234567       2023-04-13          pants  40.0              0   
3        2345678       2023-04-03           coat  80.0              0   
4        3456789       2023-04-04          pants  25.0              0   
5        4567890       2023-04-05          shirt  20.0              1   
6        5678901       2023-04-06          shoes  50.0              0   
7        5678901       2023-04-12          pants  25.0              1   
8        6789012       2023-04-07           coat  80.0              1   
9        6789012       2023-04-11           coat  80.0              0   
10       7890123       2023-04-08          pants  25.0              0   
11       8901234       2023-04-09          shirt  20.0              0   
12       9012345       2023-04-10          shoes  5

In [17]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('example.db')

# SQL query to identify the returning customers using a self-join
query = """
SELECT a.customer_num,
       a.date_of_purchase,
       a.item_purchased,
       a.cost,
       a.item_returned,
       CASE
           WHEN b.date_of_purchase IS NULL THEN 1
           ELSE 0
       END AS first_purchase_indicator
FROM sales a
LEFT JOIN sales b
ON a.customer_num = b.customer_num AND b.date_of_purchase < a.date_of_purchase
GROUP BY a.customer_num, a.date_of_purchase, a.item_purchased, a.cost, a.item_returned
"""

# Execute the query and use pandas to display the results in a table format
df = pd.read_sql_query(query, conn)
print(df)

# Close the connection
conn.close()


    customer_num date_of_purchase item_purchased  cost  item_returned  \
0        1234567       2023-04-01          shirt  20.0              0   
1        1234567       2023-04-02          shoes  50.0              1   
2        1234567       2023-04-13          pants  40.0              0   
3        2345678       2023-04-03           coat  80.0              0   
4        3456789       2023-04-04          pants  25.0              0   
5        4567890       2023-04-05          shirt  20.0              1   
6        5678901       2023-04-06          shoes  50.0              0   
7        5678901       2023-04-12          pants  25.0              1   
8        6789012       2023-04-07           coat  80.0              1   
9        6789012       2023-04-11           coat  80.0              0   
10       7890123       2023-04-08          pants  25.0              0   
11       8901234       2023-04-09          shirt  20.0              0   
12       9012345       2023-04-10          shoes  5

In [ ]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('example.db')

# SQL query using CTEs to calculate the sum paid by returning customers without considering returns
query = """
WITH ReturningCustomers AS (
    SELECT customer_num
    FROM sales
    GROUP BY customer_num
    HAVING COUNT(*) > 1
),
ValidPurchases AS (
    SELECT s.customer_num,
           s.cost
    FROM sales s
    INNER JOIN ReturningCustomers rc ON s.customer_num = rc.customer_num
    WHERE s.item_returned = 0  -- Only include non-returned items
)

SELECT customer_num, SUM(cost) AS total_paid
FROM ValidPurchases
GROUP BY customer_num
"""

# Execute the query and use pandas to display the results in a table format
df = pd.read_sql_query(query, conn)
print(df)

# Close the connection
conn.close()


   customer_num  total_paid
0       1234567        20.0
1       5678901        50.0
2       6789012        80.0
